In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## Install Libraries

In [3]:

!pip install --upgrade transformers
!pip install datasets
!pip install rouge_score
!pip install rouge
!pip install sentencepiece
!pip install Dataset
from datasets import load_dataset, load_metric, Dataset

     |████████████████████████████████| 4.0 MB 4.1 MB/s 
     |████████████████████████████████| 895 kB 82.2 MB/s 
     |████████████████████████████████| 77 kB 8.9 MB/s 
     |████████████████████████████████| 596 kB 75.6 MB/s 
     |████████████████████████████████| 6.5 MB 64.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 4.2 MB/s 
     |████████████████████████████████| 1.1 MB 67.1 MB/s 
     |████████████████████████████████| 136 kB 92.0 MB/s 
     |████████████████████████████████| 212 kB 75.5 MB/s 
     |████████████████████████████████| 127 kB 74.2 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
     |████████████████████████████████| 271 kB 93.8 MB/s 
     |████████████████████████████████| 144 kB 91.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

## Dataset

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/NN_Project/amazon_review_dataset_processed.csv"
df = pd.read_csv(path)
amazon = Dataset.from_pandas(df)
amazon.shape

(11848, 3)

In [5]:
import transformers
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from datasets import load_dataset, load_metric
import torch
import numpy as np
import torch
model_name = 'facebook/bart-large'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = transformers.BartTokenizer.from_pretrained(model_name)

model = transformers.BartForConditionalGeneration.from_pretrained(model_name).to(torch_device)

max_source_length = 512
max_target_length = 175

def preprocess_function(reviews):
  input_sequences = reviews['reviewText']
  inputs = [sequence for sequence in input_sequences]
  model_inputs = tokenizer(inputs, max_length=max_source_length, truncation=True, padding = True)
  summaries = reviews['summary']
  labels = tokenizer(summaries, max_length=max_target_length, truncation=True, padding = True)

  model_inputs['labels'] = labels['input_ids']
  return model_inputs

tokenized_amazon = amazon.map(preprocess_function, batched=True)


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/971M [00:00<?, ?B/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [6]:

NotTest_Test = tokenized_amazon.train_test_split(test_size=0.1,seed=42)
NotTest = NotTest_Test["train"]
test = NotTest_Test["test"]

Train_Val = NotTest.train_test_split(test_size=0.1,seed=42)
train = Train_Val["train"]
val = Train_Val["test"]

print(train.shape, val.shape, test.shape)

(9596, 6) (1067, 6) (1185, 6)


## Training

In [8]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir = "./results",
    evaluation_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = train,
    eval_dataset = val,
    tokenizer = tokenizer,
    data_collator = data_collator
)

trainer.train()

Using amp half precision backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, reviewText, Unnamed: 0. If summary, reviewText, Unnamed: 0 are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9596
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 28788


Epoch,Training Loss,Validation Loss
1,1.821500,1.696527
2,1.402200,1.622355
3,1.256100,1.641620


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1500/special_toke

TrainOutput(global_step=28788, training_loss=1.5654108848640682, metrics={'train_runtime': 9305.1182, 'train_samples_per_second': 3.094, 'train_steps_per_second': 3.094, 'total_flos': 3.119330364280013e+16, 'train_loss': 1.5654108848640682, 'epoch': 3.0})

In [9]:
trainer.save_model("/content/drive/MyDrive/finetunedModelBartLarge")

Saving model checkpoint to /content/drive/MyDrive/finetunedModelBartLarge
Configuration saved in /content/drive/MyDrive/finetunedModelBartLarge/config.json
Model weights saved in /content/drive/MyDrive/finetunedModelBartLarge/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/finetunedModelBartLarge/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/finetunedModelBartLarge/special_tokens_map.json


In [7]:
finetuned = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/finetunedModelBartLarge")

## Generate Summaries

In [8]:
import math
all_outputs = []
x = torch.Tensor(test['input_ids']).long()

In [9]:
outputs = finetuned.generate(x[:20], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [10]:
outputs = finetuned.generate(x[20:40], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [11]:

outputs = finetuned.generate(x[40:60], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [12]:
outputs = finetuned.generate(x[60:80], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [13]:
outputs = finetuned.generate(x[80:100], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [14]:
outputs = finetuned.generate(x[100:120], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [15]:
outputs = finetuned.generate(x[120:140], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [16]:
outputs = finetuned.generate(x[140:160], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [17]:
outputs = finetuned.generate(x[160:180], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

In [18]:
outputs = finetuned.generate(x[180:200], max_length=25, min_length=5, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
all_outputs.append(decoded_outputs)

## Evaluation

In [20]:
metric = load_metric('rouge')

def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

flattened_outputs = np.array(all_outputs).flatten()
calc_rouge_scores(test['summary'][:200], flattened_outputs)




{'rouge1': 20.1, 'rouge2': 8.4, 'rougeL': 17.9, 'rougeLsum': 17.8}

In [21]:
metric = load_metric('rouge')
flattened_outputs = np.array(all_outputs).flatten()
values = metric.compute(predictions=test['summary'][:200], references = flattened_outputs, use_stemmer=True)
values

{'rouge1': AggregateScore(low=Score(precision=0.1961832457354952, recall=0.16779749881879819, fmeasure=0.17637464991402177), mid=Score(precision=0.2231084019287386, recall=0.1906912476498389, fmeasure=0.2005141120550728), high=Score(precision=0.2551330360520328, recall=0.216154546410869, fmeasure=0.2265641035510296)),
 'rouge2': AggregateScore(low=Score(precision=0.07261498638208196, recall=0.05950313659991969, fmeasure=0.06329597904001423), mid=Score(precision=0.09500317901053193, recall=0.07887917964388551, fmeasure=0.08373584697939376), high=Score(precision=0.1226514628141466, recall=0.1006860202501746, fmeasure=0.10718531905933755)),
 'rougeL': AggregateScore(low=Score(precision=0.17238722076105, recall=0.14768394094725357, fmeasure=0.15472413525597814), mid=Score(precision=0.19945276887285404, recall=0.1703265566992346, fmeasure=0.1789572483782788), high=Score(precision=0.22921802464928437, recall=0.19225853584893343, fmeasure=0.20162475253310588)),
 'rougeLsum': AggregateScore(lo

In [22]:
df_result = pd.DataFrame({'review':test['reviewText'][:200],'generated_summaries':flattened_outputs,'target_summaries':test['summary'][:200]})
df_result.to_csv('/content/drive/MyDrive/NN_Project/bart_large.csv') 